## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
!pip install --upgrade nltk gensim bokeh

Requirement already up-to-date: nltk in /Users/kevin/anaconda3/lib/python3.6/site-packages (3.4)
Requirement already up-to-date: gensim in /Users/kevin/anaconda3/lib/python3.6/site-packages (3.6.0)
    100% |████████████████████████████████| 16.3MB 1.9MB/s eta 0:00:01
  Running setup.py bdist_wheel for bokeh ... done
  Stored in directory: /Users/kevin/Library/Caches/pip/wheels/a2/0d/1e/d5ecb93d15d5c7558fb3ac84018487fc1b24de63b2459a7822
Successfully built bokeh
  Found existing installation: bokeh 1.0.2
    Uninstalling bokeh-1.0.2:
      Successfully uninstalled bokeh-1.0.2


In [2]:
# download the data:
# !wget https://yadi.sk/i/BPQrUu1NaTduEw -O ./quora.txt
# wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

https://yadi.sk/i/BPQrUu1NaTduEw: Unsupported scheme.


In [40]:
import numpy as np

data = list(open("./quora.txt"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [6]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [7]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [[token.lower() for token in tokenizer.tokenize(data_i)] for data_i in data]# YOUR CODE

In [8]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [9]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [10]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [11]:
# now you can get word vectors !
model.get_vector('anything')

array([-0.87260324,  3.0559235 , -1.4077358 , -2.6643405 , -1.0749393 ,
        1.0228498 , -0.8680724 ,  0.0226466 ,  1.2350143 ,  0.40023595,
        1.6002399 , -0.76589495, -3.885988  ,  1.399856  ,  2.3780005 ,
        2.9223025 , -1.6998045 , -0.65331846,  2.438881  ,  1.4201504 ,
        0.5654527 , -2.1091228 ,  1.5484767 , -0.93811905, -1.174007  ,
        2.2483747 ,  2.0615473 , -0.12854108, -2.3423164 ,  2.2843819 ,
       -1.75962   , -4.646139  ], dtype=float32)

In [12]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

/Users/kevin/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('rice', 0.9627184867858887),
 ('cheese', 0.9311408996582031),
 ('fruit', 0.9299529194831848),
 ('sauce', 0.9256577491760254),
 ('potatoes', 0.9208678007125854),
 ('beans', 0.9197109937667847),
 ('butter', 0.9176644086837769),
 ('potato', 0.9149073958396912),
 ('wine', 0.914688229560852),
 ('honey', 0.9104763865470886)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [13]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [14]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

/Users/kevin/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.538511335849762),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.49642330408096313),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561611652374),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [15]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [16]:
type(words)

list

In [17]:
# for each word, compute it's vector with model
from gensim.models import Word2Vec

word_vectors = np.array([model.get_vector(words_i) for words_i in words])

In [18]:
word_vectors.shape

(1000, 100)

In [19]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [20]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance

pca=PCA(n_components=2)
new_data = pca.fit_transform(word_vectors)

word_vectors_pca = (new_data - new_data.mean(0))/ new_data.std(0)

# and maybe MORE OF YOUR CODE here :)

In [21]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [22]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [23]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [24]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca
data = TSNE(n_components=2, verbose = 100).fit_transform(word_vectors)

word_tsne = (data - data.mean(0))/data.std(0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.006s...
[t-SNE] Computed neighbors for 1000 samples in 0.177s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.034s
[t-SNE] Iteration 50: error = 68.1015930, gradient norm = 0.3242243 (50 iterations in 7.114s)
[t-SNE] Iteration 100: error = 67.7330933, gradient norm = 0.2899728 (50 iterations in 5.960s)
[t-SNE] Iteration 150: error = 69.2100983, gradient norm = 0.3044276 (50 iterations in 6.017s)
[t-SNE] Iteration 200: error = 69.3583527, gradient norm = 0.2856363 (50 iterations in 6.054s)
[t-SNE] Iteration 250: error = 69.0809174, gradient norm = 0.2981348 (50 iterations in 6.148s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.080917
[t-SNE] Iteration 300: error = 1.1834296, gradient norm = 0.0024696 (50 iterations in 4.402s)
[t-SNE] Iteration 350: error = 1.0884676, gradient norm = 0.00048

In [25]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1112', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [35]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    phrase = phrase.lower()
    counter = 0
    for word in tokenizer.tokenize(phrase):
        if word in model.vocab:
            vector += model.get_vector(word)
            counter += 1
    
    vector = vector / (counter if counter > 0 else 1)
    return vector
        
    

In [36]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [44]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [45]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [46]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.002s...
[t-SNE] Computed neighbors for 1001 samples in 0.170s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 0.477874
[t-SNE] Computed conditional probabilities in 0.032s
[t-SNE] Iteration 50: error = 78.5835495, gradient norm = 0.3215681 (50 iterations in 6.124s)
[t-SNE] Iteration 100: error = 80.0913391, gradient norm = 0.3238750 (50 iterations in 6.762s)
[t-SNE] Iteration 150: error = 81.7188721, gradient norm = 0.2901382 (50 iterations in 7.160s)
[t-SNE] Iteration 200: error = 81.5119934, gradient norm = 0.3039966 (50 iterations in 7.280s)
[t-SNE] Iteration 250: error = 82.4511261, gradient norm = 0.2837528 (50 iterations in 6.970s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 82.451126
[t-SNE] Iteration 300: error = 2.1159551, gradient norm = 0.0030566 (50 iterations in 5.478s)

In [47]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1230', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [48]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [59]:
from sklearn.metrics.pairwise import cosine_similarity




def my_cos(v1, v2):
    a = np.sqrt(np.sum(np.power(v1, 2))) * np.sqrt(np.sum(np.power(v2, 2)))
    if a == 0:
        return 0
    else:
        return ((v1.dot(v2)) / a)


def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query = get_phrase_embedding(query)
    
    data_vectors_cos = [my_cos(i, query) for i in data_vectors]
    data_vectors_cos = list(np.argsort(data_vectors_cos))
    data_vectors_cos = data_vectors_cos.reverse()[:k]

    return [data[i] for i in data_vectors_cos]

In [64]:
data_vectors[:50]

array([[ 0.04363502,  0.07049451,  0.02798353, ..., -0.02216815,
         0.14445603, -0.00921343],
       [ 0.20338728, -0.16881353,  0.06885129, ...,  0.03479549,
         0.11756916, -0.01265836],
       [ 0.23367858,  0.252785  , -0.05936207, ...,  0.05662   ,
         0.29347667,  0.20834465],
       ...,
       [ 0.14212966, -0.04426   ,  0.27257666, ..., -0.07076255,
         0.31015775,  0.13865088],
       [ 0.13847022, -0.05104311,  0.29000056, ..., -0.011886  ,
         0.19356889,  0.01433333],
       [ 0.1390783 , -0.08883129,  0.0661462 , ..., -0.0179156 ,
         0.03685818,  0.10147899]], dtype=float32)

In [63]:
my_cos(data_vectors[2], data_vectors[3])

0.081791736

In [60]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

TypeError: 'NoneType' object is not subscriptable

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.